# 1. Imports + Config

In [10]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib.colors import to_hex

from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import (
    ColumnDataSource, Div, CustomJS, HoverTool,
    LinearColorMapper, ColorBar, FixedTicker, BasicTicker
)
from bokeh.plotting import figure

output_notebook()

CSV_PATH = "temperature_daily.csv"

MONTH_NAMES = [
    "January","February","March","April","May","June",
    "July","August","September","October","November","December"
]

# Cell layout in axis units (each cell is 1x1 in plot coordinate space)
CELL_PAD = 0.10
RECT_W = 0.95
RECT_H = 0.95


Loading BokehJS ...

# 2. Load data + keep last 10 years

In [11]:
df = pd.read_csv(CSV_PATH)
df["date"] = pd.to_datetime(df["date"])
df["year"]  = df["date"].dt.year
df["month"] = df["date"].dt.month
df["day"]   = df["date"].dt.day

max_year = int(df["year"].max())
min_year = max_year - 9  # inclusive => 10 years
df10 = df[(df["year"] >= min_year) & (df["year"] <= max_year)].copy()

years = sorted(df10["year"].unique().tolist())
year_to_idx = {y: i for i, y in enumerate(years)}

print("Using years:", years[0], "to", years[-1])


Using years: 2008 to 2017


# 3. Helper functions (palette + sparkline geometry)

In [8]:
def mpl_palette(name="RdYlBu_r", n=256):
    """
    Matplotlib 3.7+ compatible: avoids deprecated cm.get_cmap.
    Returns a list of hex colors for Bokeh.
    """
    cmap = matplotlib.colormaps.get_cmap(name).resampled(n)
    return [to_hex(cmap(i)) for i in range(cmap.N)]

def normalize(vals, vmin, vmax):
    if np.isclose(vmax, vmin):
        return np.full_like(vals, 0.5, dtype=float)
    return (vals - vmin) / (vmax - vmin)

def sparkline_xy(year_idx, month_idx, days, series, local_min, local_max):
    """
    Map daily series into a 1x1 cell:
      x in [year_idx, year_idx+1], y in [month_idx, month_idx+1]
    """
    days = np.asarray(days, dtype=float)
    series = np.asarray(series, dtype=float)

    # Normalize x by day-of-month
    if len(days) <= 1:
        x_norm = np.array([0.5])
    else:
        x_norm = (days - days.min()) / (days.max() - days.min())

    y_norm = normalize(series, local_min, local_max)

    # Apply padding inside cell
    x0, x1 = year_idx + CELL_PAD, year_idx + 1 - CELL_PAD
    y0, y1 = month_idx + CELL_PAD, month_idx + 1 - CELL_PAD

    xs = (x0 + x_norm * (x1 - x0)).tolist()
    ys = (y0 + y_norm * (y1 - y0)).tolist()
    return xs, ys


# 4. Build matrix data (cells + sparklines)

In [12]:
g = df10.groupby(["year", "month"], sort=True)

cell_rows = []
spark_xs_max, spark_ys_max = [], []
spark_xs_min, spark_ys_min = [], []

for y in years:
    for m in range(1, 13):
        ym = f"{y}-{m:02d}"
        xi = year_to_idx[y]
        mi = m - 1  # 0..11

        if (y, m) not in g.groups:
            # Missing month
            cell_rows.append({
                "year": y, "month": m, "ym": ym,
                "x": xi + 0.5, "y": mi + 0.5,
                "monthly_max": np.nan,
                "monthly_min": np.nan,
                "value": np.nan,   # mode-dependent field
            })
            continue

        chunk = g.get_group((y, m)).sort_values("day")
        days = chunk["day"].to_numpy()
        dmax = chunk["max_temperature"].to_numpy(dtype=float)
        dmin = chunk["min_temperature"].to_numpy(dtype=float)

        monthly_max = float(np.nanmax(dmax))
        monthly_min = float(np.nanmin(dmin))

        cell_rows.append({
            "year": y, "month": m, "ym": ym,
            "x": xi + 0.5, "y": mi + 0.5,
            "monthly_max": monthly_max,
            "monthly_min": monthly_min,
            "value": monthly_max,   # default mode = max
        })

        # Local scaling per cell
        local_min = float(np.nanmin(dmin))
        local_max = float(np.nanmax(dmax))

        xs1, ys1 = sparkline_xy(xi, mi, days, dmax, local_min, local_max)
        xs2, ys2 = sparkline_xy(xi, mi, days, dmin, local_min, local_max)

        spark_xs_max.append(xs1); spark_ys_max.append(ys1)
        spark_xs_min.append(xs2); spark_ys_min.append(ys2)

cell_source = ColumnDataSource(pd.DataFrame(cell_rows))
max_line_source = ColumnDataSource({"xs": spark_xs_max, "ys": spark_ys_max})
min_line_source = ColumnDataSource({"xs": spark_xs_min, "ys": spark_ys_min})

print("Cells:", len(cell_source.data["ym"]), " Sparklines:", len(max_line_source.data["xs"]))


Cells: 120  Sparklines: 118


# 5. Plot (matrix + tooltip + toggle + legend)

In [13]:
global_low  = float(np.nanmin(df10["min_temperature"]))
global_high = float(np.nanmax(df10["max_temperature"]))

palette = mpl_palette("RdYlBu_r", 256)
mapper = LinearColorMapper(palette=palette, low=global_low, high=global_high, nan_color="#eeeeee")

p = figure(
    width=1100, height=720,
    x_range=(0, len(years)),
    y_range=(12, 0),  # reverse: Jan at top
    tools="tap,pan,wheel_zoom,reset,save",
    active_tap="auto",
    toolbar_location="above",
)
p.grid.visible = False
p.outline_line_color = None

# X axis at centers
x_ticks = [i + 0.5 for i in range(len(years))]
p.xaxis.ticker = FixedTicker(ticks=x_ticks)
p.xaxis.major_label_overrides = {t: str(years[i]) for i, t in enumerate(x_ticks)}
p.xaxis.axis_label = "Year"
p.xaxis.major_label_text_font_size = "10pt"

# Y axis at centers
y_ticks = [i + 0.5 for i in range(12)]
p.yaxis.ticker = FixedTicker(ticks=y_ticks)
p.yaxis.major_label_overrides = {t: MONTH_NAMES[i] for i, t in enumerate(y_ticks)}
p.yaxis.axis_label = "Month"
p.yaxis.major_label_text_font_size = "10pt"

rect_renderer = p.rect(
    x="x", y="y",
    width=RECT_W, height=RECT_H,
    source=cell_source,
    fill_color={"field": "value", "transform": mapper},
    line_color="white", line_width=2,
)

# Mini lines (daily max + daily min)
p.multi_line(xs="xs", ys="ys", source=max_line_source, line_width=1.6, line_color="#19b319")
p.multi_line(xs="xs", ys="ys", source=min_line_source, line_width=1.6, line_color="#3cc6ff")

# Tooltip uses the current 'value'
p.add_tools(HoverTool(
    renderers=[rect_renderer],
    tooltips=[("Date", "@ym"), ("Temperature", "@value{0.0} °C")]
))

colorbar = ColorBar(
    color_mapper=mapper,
    ticker=BasicTicker(desired_num_ticks=7),
    label_standoff=8,
    border_line_color=None,
    title="Monthly Max (°C)",
)
p.add_layout(colorbar, "right")

# ✅ Bokeh 3.x: use `styles` (plural), NOT `style`
badge = Div(
    text="<b>Mode:</b> Monthly Max &nbsp; (Click any cell to toggle Max/Min)",
    styles={"font-size": "14px", "margin-bottom": "8px"}
)

state = ColumnDataSource(data={"mode": ["max"]})

toggle_js = CustomJS(
    args=dict(source=cell_source, state=state, badge=badge, colorbar=colorbar),
    code="""
        const mode = state.data['mode'][0];
        const next = (mode === 'max') ? 'min' : 'max';
        state.data['mode'][0] = next;

        const data = source.data;
        const field = (next === 'max') ? 'monthly_max' : 'monthly_min';

        // Update value column => background colors + tooltip update
        data['value'] = data[field].slice();

        badge.text = `<b>Mode:</b> Monthly ${next === 'max' ? 'Max' : 'Min'} &nbsp; (Click any cell to toggle Max/Min)`;
        colorbar.title = `Monthly ${next === 'max' ? 'Max' : 'Min'} (°C)`;

        // Clear selection so repeated clicks still trigger
        source.selected.indices = [];

        source.change.emit();
        state.change.emit();
    """
)

cell_source.selected.js_on_change("indices", toggle_js)

show(column(badge, p))
